# Palpite
Cartola FC tips.

## Goal
To develop a model to predict how many points will an athlete score on Cartola FC.

The model is planned to be deployd on a web app and shall only use information available on the official Cartola FC API and on any popular betting API.

## Data Sources
### Cartola FC
The historical Cartola FC dataset used on this model is available on [github.com/henriquepgomide/caRtola](https://github.com/henriquepgomide/caRtola/tree/master/data).

### Betting Lines
The historical betting lines for Brasileirão Série A is available on [football-data.co.uk/brazil.php](https://www.football-data.co.uk/brazil.php).

Many thanks to the maintainers from both projects on making the data available for free to the world.

## Time Interval
The official Cartola FC API changed its format on 2018. Datasets before this year do not attend the expected formats, and therefore, won't be used.

The seasons used to build the model will be:

* 38 2018 rounds (complete season)
* 38 2019 rounds (complete season)
* 18 2020 rounds (almost half season)

It means:

* 2.5 seaons
* 94 rounds
* 940 matches

It seems to be enough and there is no need of trying to format previous years' datasets.

## Imports
Please make sure you have all packages installed before starting using the notebook.

In [1]:
import datetime
import itertools
import joblib
import os
import pytz

import numpy as np
import pandas as pd
import tpot
import sklearn.metrics
import sklearn.model_selection

c:\users\mcouto\desktop\personal\palpite\venv\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


## Data Wrangling
### Load Cartola FC data
Load data from 2018, 2019 and 2020 Cartola FC seasons.

In [2]:
def load_all_rounds(season):
    """ 
    Load all rounds data from a season to a pandas DataFrame.
    
    Also rename columns to a shorter name and introduce a year feature.
    """
    # Define file paths.
    file_path_list = [f"data//cartola//{season}//rodada-{i + 1}.csv" for i in range(38)]
    # Load datasets into a list.
    rounds = [
        pd.read_csv(file_path, index_col=0) for file_path in file_path_list if os.path.exists(file_path)
        ]
    # Concatenate the list.
    data = pd.concat(rounds, ignore_index=True)

    # Remove and repetive term from columns names.
    data.columns = [col.replace("atletas.", "") for col in data.columns]

    # Create year column.
    data["year"] = [season for _ in range(data.shape[0])]

    return data


def load_seasons(seasons):
    """ Load specified seasons into a same pandas DataFrame. """
    return pd.concat([load_all_rounds(year) for year in seasons], ignore_index=True)
    

cartola_data = load_seasons([2018, 2019, 2020])
cartola_data.head()

,nome,slug,apelido,foto,atleta_id,rodada_id,clube_id,posicao_id,status_id,pontos_num,...,FT,GS,CV,GC,PP,DP,year,jogos_num,PI,DS
0,Matheus Ferraz Pereira,matheus-ferraz,Matheus Ferraz,https://s.glbimg.com/es/sde/f/2018/03/17/6d461...,38632,1,AME,zag,Nulo,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN
1,Willian Lanes de Lima,lima,Lima,https://s.glbimg.com/es/sde/f/2018/03/17/3d9ef...,38506,1,AME,zag,Nulo,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN
2,Rómulo Otero Vásquez,otero,Otero,https://s.glbimg.com/es/sde/f/2017/04/03/9fe40...,83004,1,ATL,mei,Provável,16.5,...,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN
3,Diego Ribas da Cunha,diego,Diego,https://s.glbimg.com/es/sde/f/2017/08/16/3ba37...,38909,1,FLA,mei,Provável,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN
4,Rodrigo Eduardo Costa Marinho,rodriguinho,Rodriguinho,https://s.glbimg.com/es/sde/f/2018/03/20/c125f...,61033,1,COR,mei,Provável,16.5,...,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN


### Number of Played Games
Most recent year come with a number o played gamed feature, however, 2018 doesn't have it. I'll recreate it for all years.

In [3]:
# If points are different than zero, the player played the game.
cartola_data["played"] = (cartola_data["pontos_num"] != 0).astype(int)

# Sort values, group by athlete and season and the calculate the cumulative sum of played games.
cartola_data = cartola_data.sort_values(["atleta_id", "year", "rodada_id"])
cartola_data["matches"] = cartola_data.groupby(["atleta_id", "year"]).cumsum()["played"]
cartola_data.head()

,nome,slug,apelido,foto,atleta_id,rodada_id,clube_id,posicao_id,status_id,pontos_num,...,CV,GC,PP,DP,year,jogos_num,PI,DS,played,matches
372,Juan Silveira dos Santos,juan,Juan,https://s.glbimg.com/es/sde/f/2017/08/16/8083b...,36540,1,FLA,zag,Provável,0.5,...,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,1,1
1109,Juan Silveira dos Santos,juan,Juan,https://s.glbimg.com/es/sde/f/2017/08/16/8083b...,36540,2,FLA,zag,Dúvida,0.0,...,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,0,1
1586,Juan Silveira dos Santos,juan,Juan,https://s.glbimg.com/es/sde/f/2018/05/04/a27fb...,36540,3,FLA,zag,Provável,5.0,...,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,1,2
2379,Juan Silveira dos Santos,juan,Juan,https://s.glbimg.com/es/sde/f/2018/05/04/a27fb...,36540,4,FLA,zag,Provável,0.0,...,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,0,2
3172,Juan Silveira dos Santos,juan,Juan,https://s.glbimg.com/es/sde/f/2018/05/04/a27fb...,36540,5,FLA,zag,Contundido,2.3,...,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,1,3


### Keep relevant data
This model do not aim to use athletes scouts (stats). They are all removed. 

Redundant features, like athlete id, name, nickname, slug are also removed.

In [4]:
to_keep = [
    'year',  # Number of years played
    'apelido',  # Athlete nickname.
    'rodada_id',  # Round number.
    'clube.id.full.name',  # Club full name.
    'posicao_id',  # Position ID.
    'status_id',  # Status ID.
    'media_num',  # Mean points.
    'preco_num',  # Price value.
    'variacao_num',  # Price variation.
    'matches',  # Number of matches played.
    'pontos_num',  # Points.
]

cartola_data = cartola_data[to_keep]
cartola_data.tail()

,year,apelido,rodada_id,clube.id.full.name,posicao_id,status_id,media_num,preco_num,variacao_num,matches,pontos_num
74568,2020,Igor Torres,18,356,ata,Nulo,0.0,1.00,0.00,0,0.0
73816,2020,Sandrinho,17,290,ata,Nulo,0.0,1.00,0.00,0,0.0
74576,2020,Sandrinho,18,290,ata,Nulo,-3.0,0.77,-0.23,1,-3.0
74577,2020,Ângelo,18,Santos,ata,Nulo,0.8,1.15,0.15,1,0.8
74575,2020,Brayan,18,Santos,ata,Nulo,0.0,1.00,0.00,0,0.0


### Translate
Code should preferable be written in English. 


Even though it is a brazilian fantasy game, with a brazilian target audience, 
you never know when someone else may be interested in reading your code for whatever the purpose it may be. 
So it is a nice idea to always keep it in English.

In [5]:
# Rename columns.
cartola_data = cartola_data.rename({
    "apelido": "name",
    "rodada_id": "round",
    "clube.id.full.name": "club",
    "posicao_id": "position",
    "status_id": "status",
    "pontos_num": "points",
    "media_num": "mean",
    "preco_num": "price",
    "variacao_num": "variation",
}, axis=1)

# Rename values from the column position.
cartola_data["position"] = cartola_data["position"].replace({
    "zag": "defender",
    "mei": "midfielder",
    "ata": "forward",
    "gol": "goalkeeper",
    "lat": "fullback",
    "tec": "coach",
})

# Rename values from the column status.
cartola_data["status"] = cartola_data["status"].replace({
    "Nulo": "null",
    "Provável": "expected",
    "Contundido": "injured",
    "Dúvida": "doubt",
    "Suspenso": "suspended",
})

cartola_data.head()

,year,name,round,club,position,status,mean,price,variation,matches,points
372,2018,Juan,1,Flamengo,defender,expected,0.50,8.61,-3.39,1,0.5
1109,2018,Juan,2,Flamengo,defender,doubt,0.50,8.61,0.00,1,0.0
1586,2018,Juan,3,Flamengo,defender,expected,2.75,7.83,-0.78,2,5.0
2379,2018,Juan,4,Flamengo,defender,expected,2.75,7.83,0.00,2,0.0
3172,2018,Juan,5,Flamengo,defender,injured,2.60,7.20,-0.63,3,2.3


### Clubs
It looks like there is pertubation in the `club` column.

Let's further invertigate it and decide how to fix it.

In [6]:
cartola_data["club"].unique()

array(['Flamengo', 'Avaí', 'Fluminense', 'Botafogo', '263', '293',
       'Bahia', 'Chapecoense', 'Ceará', '354', 'Goiás', 'Cruzeiro',
       'Palmeiras', '265', 'CSA', 'Atlético-PR', 'Athlético-PR', 'Sport',
       'Santos', 'Vasco', 'América-MG', '290', 'Grêmio', 'Atlético-MG',
       'Vitória', 'Fortaleza', '356', 'Internacional', 'Paraná',
       'Corinthians', 'Bragantino', '280', '264', '266', '284', '282',
       '292', 'Coritiba', '267', '275', 'São Paulo', '276', 'Atlético-GO',
       '373', '285'], dtype=object)

Some clubs names are represeted by its full name, while others are using the ID. To be coherent, I'll convert those IDs into names.

Since the names are case sensitive and have special characters that may produce errors later on, why not use IDs only? 
It is because the ID system changed throughout the years. It means that a same club may have different IDs for the year 2018 and 2019. It could produce erros that are harder to debug.

Even though full names are also prone to error, they are easier to debug, since they are written in natural language.

In order to correlate IDs to names, first it is important to understant in which year it is happening to use the most appropiate mapping.


In [7]:

def is_int(value):
    """ Check if a value is integer. """
    try:
        int(value)
        return True
    except ValueError:
        return False


def count_int(sequence):
    """ Count the amount of integers in a sequence. """
    return len([value for value in sequence if is_int(value)])


print(f"There are {count_int(cartola_data[cartola_data['year'] == 2018]['club'])} IDs in the year 2018")
print(f"There are {count_int(cartola_data[cartola_data['year'] == 2019]['club'])} IDs in the year 2019")
print(f"There are {count_int(cartola_data[cartola_data['year'] == 2020]['club'])} IDs in the year 2020")

There are 0 IDs in the year 2018
There are 0 IDs in the year 2019
There are 4396 IDs in the year 2020


All of the IDs are from the 2020 year. So I'll use the mapping from this year. The clubs dataset was retrieved from the official Cartola FC API.

In [8]:
clubs_data = pd.read_csv(r"data\cartola\clubes.csv", index_col=0)
clubs_data.head()

,id,nome,abreviacao,escudos,nome_fantasia
1,1,Outros,OUT,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Outros
1349,1349,Ipatinga,IPA,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Ipatinga
1371,1371,Cuiabá,CUI,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Cuiabá
1390,1390,Icasa,ICA,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Icasa
2190,2190,Oeste,OES,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Oeste


#### Translate Clubs Dataset
For the same reason as before, the dataset needs to be translated.

In [9]:
# Rename columns.
clubs_data = clubs_data.rename({
    "nome": "name",
    "abreviacao": "abbreviation",
    "escudos": "logos",
    "nome_fantasia": "fanstasy_name",
}, axis=1)

clubs_data.head()

,id,name,abbreviation,logos,fanstasy_name
1,1,Outros,OUT,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Outros
1349,1349,Ipatinga,IPA,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Ipatinga
1371,1371,Cuiabá,CUI,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Cuiabá
1390,1390,Icasa,ICA,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Icasa
2190,2190,Oeste,OES,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Oeste


In [10]:
def apply_mapping(value, mapping):
    """ Apply a mapping to column values. """
    if value in mapping:
        return mapping[value]
    return value



id_to_name_mapping = {str(row.id): row.name for row in clubs_data.itertuples()}

cartola_data["club"] = cartola_data["club"].apply(apply_mapping, mapping=id_to_name_mapping)
cartola_data["club"].unique()

array(['Flamengo', 'Avaí', 'Fluminense', 'Botafogo', 'Athlético-PR',
       'Bahia', 'Chapecoense', 'Ceará', 'Goiás', 'Cruzeiro', 'Palmeiras',
       'CSA', 'Atlético-PR', 'Sport', 'Santos', 'Vasco', 'América-MG',
       'Grêmio', 'Atlético-MG', 'Vitória', 'Fortaleza', 'Internacional',
       'Paraná', 'Corinthians', 'Bragantino', 'Coritiba', 'São Paulo',
       'Atlético-GO'], dtype=object)

We are almost done with the clubs. The last thing that needs to be fixed is the name from Athlético-PR.

This team changes its name in recent years and because of it, is duplicated.

In [11]:
cartola_data["club"] = cartola_data["club"].replace("Atlético-PR", "Athlético-PR")
cartola_data["club"].unique()

array(['Flamengo', 'Avaí', 'Fluminense', 'Botafogo', 'Athlético-PR',
       'Bahia', 'Chapecoense', 'Ceará', 'Goiás', 'Cruzeiro', 'Palmeiras',
       'CSA', 'Sport', 'Santos', 'Vasco', 'América-MG', 'Grêmio',
       'Atlético-MG', 'Vitória', 'Fortaleza', 'Internacional', 'Paraná',
       'Corinthians', 'Bragantino', 'Coritiba', 'São Paulo',
       'Atlético-GO'], dtype=object)

### Data Transformation
There is still one big problem.

The following columns vary from game to game. 

* `status`
* `points`
* `mean`
* `matches`

Status is available before the match to the user, while mean is available after the match. In the next round the user will have status for the next match, and the mean from the last match.

Therefore, for a model to be useful for the user, it should consider next match status, and last match mean and matches.

Naturally, for the first match, mean will be null. Also, Notice that points won't be transformed since it is the target feature.

In [12]:
# Duplicate data and reduce the round feature
antecipated_data = cartola_data.copy()
antecipated_data["round"] = [round_ - 1 for round_ in antecipated_data["round"]]

# Join on features that don't change
join_on = ['year', 'name', 'round', 'club', 'position']
cartola_data = pd.merge(
    left=antecipated_data, 
    right=cartola_data, 
    on=join_on, 
    how="left", 
    suffixes=("_current", "_previous")  # Translate to _current, _previous
    )

# Reset rounds.
cartola_data["round"] = [round + 1 for round in cartola_data["round"]]

# Remove columns that won't be no longer used.
cartola_data = cartola_data.drop(
    columns=[
        "status_previous", 
        "points_previous", 
        "mean_current", 
        "price_current",
        "variation_current",
        "matches_current",
        ]
    )

# Remove suffixes
cartola_data.columns = [col.replace("_previous", "").replace("_current", "") for col in cartola_data.columns]

# Transform mean and matches NaN in 0
cartola_data["mean"] = cartola_data["mean"].fillna(0)
cartola_data["matches"] = cartola_data["matches"].fillna(0)

cartola_data.head()

,year,name,round,club,position,status,points,mean,price,variation,matches
0,2018,Juan,1,Flamengo,defender,expected,0.5,0.00,NaN,NaN,0.0
1,2018,Juan,2,Flamengo,defender,doubt,0.0,0.50,8.61,-3.39,1.0
2,2018,Juan,3,Flamengo,defender,expected,5.0,0.50,8.61,0.00,1.0
3,2018,Juan,4,Flamengo,defender,expected,0.0,2.75,7.83,-0.78,2.0
4,2018,Juan,5,Flamengo,defender,injured,2.3,2.75,7.83,0.00,2.0


### Dates
We should introduce a dates feature. This is important in order to merge with the betting lines dataset later on.

I'll concatenate the matches datasets from each year.

In [13]:
def load_date_dataset(year):
    """ Load dates dataset. """
    data = pd.read_csv(f"data\\cartola\\{year}\\{year}_partidas.csv")
    data["year"] = [year for _ in range(data.shape[0])]
    return data


# Load dates datasets.
years = [2018, 2019, 2020]
matches_data = pd.concat([load_date_dataset(year) for year in years], ignore_index=True)

print(f"There are {count_int(matches_data[matches_data['year'] == 2018]['home_team'])} IDs in the year 2018")
print(f"There are {count_int(matches_data[matches_data['year'] == 2019]['home_team'])} IDs in the year 2019")
print(f"There are {count_int(matches_data[matches_data['year'] == 2020]['home_team'])} IDs in the year 2020")

matches_data.head()

There are 0 IDs in the year 2018
There are 380 IDs in the year 2019
There are 180 IDs in the year 2020


,game,round,date,home_team,score,away_team,arena,year,home_score,away_score
0,1.0,1,14/04/2018 - 16:00,Cruzeiro - MG,0 x 1,Grêmio - RS,Mineirão - Belo Horizonte - MG,2018,NaN,NaN
1,2.0,1,15/04/2018 - 19:00,Atlético - PR,5 x 1,Chapecoense - SC,Arena da Baixada - Curitiba - PR,2018,NaN,NaN
2,3.0,1,15/04/2018 - 11:00,América - MG,3 x 0,Sport - PE,Independência - Belo Horizonte - MG,2018,NaN,NaN
3,4.0,1,14/04/2018 - 19:00,Vitória - BA,2 x 2,Flamengo - RJ,Manoel Barradas - Salvador - BA,2018,NaN,NaN
4,5.0,1,15/04/2018 - 16:00,Vasco da Gama - RJ,2 x 1,Atlético - MG,São Januário - Rio de Janeiro - RJ,2018,NaN,NaN


This dataset also have some problems that should be fixed:

* Dates are in different format depending on the year.
* Team names are in various formats.

To fix teams names I'll use the `time_ids.csv` dataset. It as some other teams names mapping.


In [14]:
alternative_clubs_data = pd.read_csv(r"data\cartola\times_ids.csv")
alternative_clubs_data.head()

,nome.cbf,nome.cartola,nome.completo,cod.older,cod.2017,cod.2018,id,abreviacao,escudos.60x60,escudos.45x45,escudos.30x30
0,América - MG,América-MG,America MG,327,327,327,327,AME,https://s.glbimg.com/es/sde/f/organizacoes/201...,https://s.glbimg.com/es/sde/f/organizacoes/201...,https://s.glbimg.com/es/sde/f/organizacoes/201...
1,America - RN,Atlético-RN,America RN,200,200,1,200,OUT,NaN,NaN,NaN
2,Atlético - GO,Atlético-GO,Atletico GO,201,373,373,373,ATL,NaN,NaN,NaN
3,Atlético - MG,Atlético-MG,Atletico Mineiro,282,282,282,282,ATL,https://s.glbimg.com/es/sde/f/equipes/2017/11/...,https://s.glbimg.com/es/sde/f/equipes/2017/11/...,https://s.glbimg.com/es/sde/f/equipes/2017/11/...
4,Atlético - PR,Atlético-PR,Atletico Paranaense,293,293,293,293,ATL,https://s.glbimg.com/es/sde/f/equipes/2015/06/...,https://s.glbimg.com/es/sde/f/equipes/2015/06/...,https://s.glbimg.com/es/sde/f/equipes/2015/06/...


In [15]:
# Translate
alternative_clubs_data = alternative_clubs_data.rename({
    "nome.cbf": "cbf_name",  # CBF is the brazilian soccer confederation.
    "nome.cartola": "cartola_name",  # Cartola FC name.
    "nome.completo": "full_name",
    "cod_older": "id_old",  # ID as before 2017.
    "cod.2017": "id_2017",  # ID as 2017
    "cod.2018": "id_2018",  # ID as 2018
    "id": "id_2019",  # ID as 2019
    "abreviacao": "abbreviation",
    "escudo.60x60": "logo_60x60",
    "escudo.45x45": "logo_45x45",
    "escudo.30x30": "logo_30x30",
}, axis=1)

alternative_clubs_data["cartola_name"] = alternative_clubs_data["cartola_name"].replace(
    "Atlético-PR", "Athlético-PR"
    )

alternative_clubs_data.head()

,cbf_name,cartola_name,full_name,cod.older,id_2017,id_2018,id_2019,abbreviation,escudos.60x60,escudos.45x45,escudos.30x30
0,América - MG,América-MG,America MG,327,327,327,327,AME,https://s.glbimg.com/es/sde/f/organizacoes/201...,https://s.glbimg.com/es/sde/f/organizacoes/201...,https://s.glbimg.com/es/sde/f/organizacoes/201...
1,America - RN,Atlético-RN,America RN,200,200,1,200,OUT,NaN,NaN,NaN
2,Atlético - GO,Atlético-GO,Atletico GO,201,373,373,373,ATL,NaN,NaN,NaN
3,Atlético - MG,Atlético-MG,Atletico Mineiro,282,282,282,282,ATL,https://s.glbimg.com/es/sde/f/equipes/2017/11/...,https://s.glbimg.com/es/sde/f/equipes/2017/11/...,https://s.glbimg.com/es/sde/f/equipes/2017/11/...
4,Atlético - PR,Athlético-PR,Atletico Paranaense,293,293,293,293,ATL,https://s.glbimg.com/es/sde/f/equipes/2015/06/...,https://s.glbimg.com/es/sde/f/equipes/2015/06/...,https://s.glbimg.com/es/sde/f/equipes/2015/06/...


In [16]:
# Create a dictionary from the dataset.
cbf_to_cartola_mapping = {
    row.cbf_name: row.cartola_name for row in alternative_clubs_data.itertuples()
    }
id_to_name_mapping_2019 = {
    row.id_2019: row.cartola_name for row in alternative_clubs_data.itertuples()
    }
id_to_name_mapping_2020 = {int(row.id): row.name for row in clubs_data.itertuples()}

# Apply CBF name mapping function.
year_to_mapping = (
    ([2018, 2019, 2020], cbf_to_cartola_mapping,),
    ([2019], id_to_name_mapping_2019,),
    ([2020], id_to_name_mapping_2020,),
)

for years, mapping in year_to_mapping:
    # Filter.
    matches_data_year = matches_data[matches_data["year"].isin(years)]
    
    # Map.
    matches_data_year["home_team"] = matches_data_year["home_team"].apply(
        apply_mapping, mapping=mapping
        )
    matches_data_year["away_team"] = matches_data_year["away_team"].apply(
        apply_mapping, mapping=mapping
        )

    # Apply.
    matches_data[matches_data["year"].isin(years)] = matches_data_year

matches_data["home_team"].unique()

c:\users\mcouto\desktop\personal\palpite\venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\mcouto\desktop\personal\palpite\venv\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


array(['Cruzeiro', 'Athlético-PR', 'América-MG', 'Vitória', 'Vasco',
       'Botafogo', 'São Paulo', 'Santos', 'Corinthians', 'Internacional',
       'Atlético-MG', 'Paraná', 'Chapecoense', 'Bahia', 'Fluminense',
       'Flamengo', 'Ceará-SC', 'Sport', 'Palmeiras', 'Grêmio', 'Avaí',
       'CSA', 'Fortaleza', 'Goiás', 'Coritiba', 'Bragantino', 'Ceará',
       'Atlético-GO'], dtype=object)

Ceará appears twice as *Ceará-SC* and *Ceará*. It should be fixed.

In [17]:
matches_data["home_team"] = matches_data["home_team"].replace("Ceará-SC", "Ceará")
matches_data["away_team"] = matches_data["away_team"].replace("Ceará-SC", "Ceará")

matches_data["home_team"].unique()

array(['Cruzeiro', 'Athlético-PR', 'América-MG', 'Vitória', 'Vasco',
       'Botafogo', 'São Paulo', 'Santos', 'Corinthians', 'Internacional',
       'Atlético-MG', 'Paraná', 'Chapecoense', 'Bahia', 'Fluminense',
       'Flamengo', 'Ceará', 'Sport', 'Palmeiras', 'Grêmio', 'Avaí', 'CSA',
       'Fortaleza', 'Goiás', 'Coritiba', 'Bragantino', 'Atlético-GO'],
      dtype=object)

We're done with teams, now is time to clean up dates.

In [18]:
def interpret_date(string, patterns, timezone=None):
    """ Interpret some specific patterns of date. """
    for pattern in patterns:
        try:
            dt = datetime.datetime.strptime(string, pattern)
            if timezone:
                dt = dt + datetime.timedelta(hours=timezone)
            return pd.to_datetime(dt.date())
        except ValueError:
            pass
    raise ValueError("Couldn't match any date pattern.")


# Interpret string ad datetime.
matches_data["date"] = matches_data["date"].apply(
    interpret_date, patterns=["%d/%m/%Y - %H:%M", "%Y-%m-%d"])

matches_data.head(5)

,game,round,date,home_team,score,away_team,arena,year,home_score,away_score
0,1.0,1,2018-04-14,Cruzeiro,0 x 1,Grêmio,Mineirão - Belo Horizonte - MG,2018,NaN,NaN
1,2.0,1,2018-04-15,Athlético-PR,5 x 1,Chapecoense,Arena da Baixada - Curitiba - PR,2018,NaN,NaN
2,3.0,1,2018-04-15,América-MG,3 x 0,Sport,Independência - Belo Horizonte - MG,2018,NaN,NaN
3,4.0,1,2018-04-14,Vitória,2 x 2,Flamengo,Manoel Barradas - Salvador - BA,2018,NaN,NaN
4,5.0,1,2018-04-15,Vasco,2 x 1,Atlético-MG,São Januário - Rio de Janeiro - RJ,2018,NaN,NaN


### Merge Cartola FC and matches datasets
Now that everything looks fine with the matches dataset, it is ready to be merged on the Cartola FC dataset.

In [19]:
# Rename the matches dataset to attend the cartola dataset features names. This is required for merging.
matches_renamed = matches_data.rename({
    "home_team": "club_home",
    "away_team": "club_away"
    }, axis=1)

# We need to do it in two steps. First merge searching for the club in home teams. 
# Then do it again for away teams.

for club_type in ["home", "away"]:
    matches_renamed["club"] = matches_renamed[f"club_{club_type}"]

    # Filter to merge only desired features.
    matches_renamed_filtered = matches_renamed[["round", "year", "club", "date"]]

    merge_on = ["round", "year", "club"]
    cartola_data = pd.merge(cartola_data, matches_renamed_filtered, on=merge_on, how="left")

# Unify both new columns in a single column.
cartola_data["date"] = np.where(
    pd.isna(cartola_data["date_x"]),
    cartola_data["date_y"],
    cartola_data["date_x"]
    )

# Remove date_x and date_y.
cartola_data = cartola_data.drop(columns=["date_x", "date_y"])

cartola_data.tail()

,year,name,round,club,position,status,points,mean,price,variation,matches,date
74757,2020,Igor Torres,18,Fortaleza,forward,null,0.0,0.0,1.0,0.0,0.0,2020-10-24
74758,2020,Sandrinho,17,Goiás,forward,null,0.0,0.0,NaN,NaN,0.0,2020-10-19
74759,2020,Sandrinho,18,Goiás,forward,null,-3.0,0.0,1.0,0.0,0.0,2020-10-24
74760,2020,Ângelo,18,Santos,forward,null,0.8,0.0,NaN,NaN,0.0,2020-10-25
74761,2020,Brayan,18,Santos,forward,null,0.0,0.0,NaN,NaN,0.0,2020-10-25


Check if all columns received a date value.

In [20]:
# Count NaNs in each row.
cartola_data.shape[0] - cartola_data.count()

year            0
name            0
round           0
club            0
position        0
status          0
points          0
mean            0
price        3142
variation    3142
matches         0
date          458
dtype: int64

There are still 458 rows without date. They needed to be investigated.

In [21]:
cartola_data[pd.isna(cartola_data["date"])].groupby(["year", "round", "club"]).first()

name    position     status  points  mean  \
year round club                                                                
2020 1     Atlético-GO   Vagner Mancini       coach   expected     0.0  0.00   
           Bahia               Anderson  goalkeeper       null     0.0  0.00   
           Botafogo       Paulo Autuori       coach   expected     0.0  0.00   
           Corinthians               Jô     forward   expected     0.0  0.00   
     11    Athlético-PR        Jonathan    fullback   expected     0.0  2.72   
           Bahia           Mano Menezes       coach   expected     0.0  0.77   
           Flamengo         Diego Alves  goalkeeper    injured     0.0  2.22   
           Goiás           Rafael Moura     forward   expected     0.0  0.00   
           São Paulo           Hernanes  midfielder      doubt     0.0  3.14   
     12    Sport           Thiago Neves  midfielder   expected     0.0  0.10   
     18    Corinthians               Jô     forward       null     0.0  2.51   
           Vasco         Leandro Castan    defender  suspended     0.0  3.22   

                         price  variation  matches date  
year round club                                          
2020 1     Atlético-GO     NaN        NaN      0.0  NaT  
           Bahia           NaN        NaN      0.0  NaT  
           Botafogo        NaN        NaN      0.0  NaT  
           Corinthians     NaN        NaN      0.0  NaT  
     11    Athlético-PR   9.85       0.98      5.0  NaT  
           Bahia          6.60      -1.46      1.0  NaT  
           Flamengo      10.72       0.00      5.0  NaT  
           Goiás          6.36       0.46      0.0  NaT  
           São Paulo      8.55      -0.29      5.0  NaT  
     12    Sport          3.66       0.00      4.0  NaT  
     18    Corinthians   10.67       0.00     12.0  NaT  
           Vasco         12.58      -0.84     11.0  NaT

It looks likes those are games that really didn't happen. 2020 was a year with a lot of delayed games due to the COVID-19 pandemic.

In [22]:
cartola_data = cartola_data.dropna()

# Count NaNs in each row.
cartola_data.shape[0] - cartola_data.count()

year         0
name         0
round        0
club         0
position     0
status       0
points       0
mean         0
price        0
variation    0
matches      0
date         0
dtype: int64

### Load betting lines data
Load betting lines from 2018, 2019 and 2020 seasons.

In [23]:
bet_data = pd.read_csv(r"data\betting\historical_betting_lines.csv")
bet_data.head()

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,Brazil,Serie A,2012,19/05/2012,22:30,Palmeiras,Portuguesa,1.0,1.0,D,1.75,3.86,5.25,1.76,3.87,5.31,1.69,3.50,4.90
1,Brazil,Serie A,2012,19/05/2012,22:30,Sport Recife,Flamengo RJ,1.0,1.0,D,2.83,3.39,2.68,2.83,3.42,2.70,2.59,3.23,2.58
2,Brazil,Serie A,2012,20/05/2012,01:00,Figueirense,Nautico,2.0,1.0,H,1.60,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64
3,Brazil,Serie A,2012,20/05/2012,20:00,Botafogo RJ,Sao Paulo,4.0,2.0,H,2.49,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84
4,Brazil,Serie A,2012,20/05/2012,20:00,Corinthians,Fluminense,0.0,1.0,A,1.96,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89


In [24]:
# Filter seasons.
bet_data = bet_data[bet_data["Season"].isin([2018, 2019, 2020])]
bet_data.head()

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
2280,Brazil,Serie A,2018,14/04/2018,20:00,Cruzeiro,Gremio,0.0,1.0,A,1.96,3.29,4.87,2.11,3.30,4.87,1.98,3.16,4.15
2281,Brazil,Serie A,2018,14/04/2018,23:00,Vitoria,Flamengo RJ,2.0,2.0,D,3.56,3.41,2.22,3.75,3.42,2.24,3.40,3.31,2.14
2282,Brazil,Serie A,2018,15/04/2018,01:00,Santos,Ceara,2.0,0.0,H,1.57,4.13,6.76,1.60,4.14,7.10,1.55,3.88,6.12
2283,Brazil,Serie A,2018,15/04/2018,15:00,America MG,Sport Recife,3.0,0.0,H,1.98,3.35,4.51,2.05,3.42,4.51,1.96,3.27,4.01
2284,Brazil,Serie A,2018,15/04/2018,20:00,Corinthians,Fluminense,2.0,1.0,H,1.67,3.68,6.55,1.68,3.74,7.00,1.62,3.55,6.03


### Keep relevant rows
We are only interested in information that is available before the game.

We take as premise that the betting odds are accurate, and we shall not try to predict whenever they will fail. This would make the model over-complicated.

With that in mind, we only need the odds and the minimum information required to merge them with the Cartola FC dataset.

#### About the odds
We will use average odds instead of maximum or from a single source to avoid outliers or bad decisions by a specific booker.

In [25]:
to_keep = ["Season", "Date", "Time", "Home", "Away", "AvgH", "AvgD", "AvgA"]
bet_data = bet_data[to_keep]
bet_data.head()

,Season,Date,Time,Home,Away,AvgH,AvgD,AvgA
2280,2018,14/04/2018,20:00,Cruzeiro,Gremio,1.98,3.16,4.15
2281,2018,14/04/2018,23:00,Vitoria,Flamengo RJ,3.40,3.31,2.14
2282,2018,15/04/2018,01:00,Santos,Ceara,1.55,3.88,6.12
2283,2018,15/04/2018,15:00,America MG,Sport Recife,1.96,3.27,4.01
2284,2018,15/04/2018,20:00,Corinthians,Fluminense,1.62,3.55,6.03


### Fit data format into Cartola FC dataset standards

Everything is in a different format related to the Cartola FC dataset. For a succesful merge this dataset needs to be reformatted.

In [26]:
# Concatenate Date and Time
bet_data["Date"] = [f"{row.Date} {row.Time}" for row in bet_data.itertuples()]
bet_data = bet_data.drop(columns=["Time"])

# Rename columns.
bet_data = bet_data.rename({
    "Season": "year",
    "Date": "date",
    "Home": "home",
    "Away": "away",
    "AvgH": "home_odds",
    "AvgD": "draw_odds",
    "AvgA": "away_odds",
}, axis=1)

# Interpret string ad datetime.
bet_data["date"] = bet_data["date"].apply(
    interpret_date, patterns=["%d/%m/%Y %H:%M"], timezone=-4,
    )

bet_data.head()

,year,date,home,away,home_odds,draw_odds,away_odds
2280,2018,2018-04-14,Cruzeiro,Gremio,1.98,3.16,4.15
2281,2018,2018-04-14,Vitoria,Flamengo RJ,3.40,3.31,2.14
2282,2018,2018-04-14,Santos,Ceara,1.55,3.88,6.12
2283,2018,2018-04-15,America MG,Sport Recife,1.96,3.27,4.01
2284,2018,2018-04-15,Corinthians,Fluminense,1.62,3.55,6.03


And now for something completely different... fix clubs names.

This time is more challenging because it doesn't seem to match any pattern used before. I will have to carefully create a new mapping.

In [27]:
bet_data["home"].unique()

array(['Cruzeiro', 'Vitoria', 'Santos', 'America MG', 'Corinthians',
       'Internacional', 'Vasco', 'Atletico-PR', 'Botafogo RJ',
       'Sao Paulo', 'Bahia', 'Flamengo RJ', 'Parana', 'Atletico-MG',
       'Ceara', 'Chapecoense-SC', 'Fluminense', 'Palmeiras', 'Gremio',
       'Sport Recife', 'Athletico-PR', 'CSA', 'Avai', 'Fortaleza',
       'Goias', 'Coritiba', 'Bragantino', 'Atletico GO'], dtype=object)

To be corrected:

* No accentuation. 
* Some teams like Botafogo and Flamengo have the states or city appended to end.
* Some teams uses dashs and other spaces.

In [28]:
teams_names_mapping = {
    "Vitoria": "Vitória",
    "America MG": "América-MG",
    "Sao Paulo": "São Paulo",
    "Flamengo RJ": "Flamengo",
    "Botafogo RJ": "Botafogo",
    "Parana": "Paraná",
    "Atletico-MG": "Atlético-MG",
    "Ceara": "Ceará",
    "Chapecoense-SC": "Chapecoense",
    "Gremio": "Grêmio",
    "Sport Recife": "Sport",
    "Athletico-PR": "Athlético-PR",
    "Atletico-PR": "Athlético-PR",
    "Avai": "Avaí",
    "Goias": "Goiás",
    "Atletico GO": "Atlético-GO",

}

# Apply mapping.
for col in ["home", "away"]:
    bet_data[col] = bet_data[col].apply(apply_mapping, mapping=teams_names_mapping)

    # Check that new columns names are the same from Cartola FC dataset.
    if all([team in cartola_data["club"].unique() for team in bet_data[col].unique()]):
        print(f"Formatting was succesfull on column {col}.")
    else:
        print(f"Something wrong with column {col} formatting.")

bet_data.head()

Formatting was succesfull on column home.
Formatting was succesfull on column away.


,year,date,home,away,home_odds,draw_odds,away_odds
2280,2018,2018-04-14,Cruzeiro,Grêmio,1.98,3.16,4.15
2281,2018,2018-04-14,Vitória,Flamengo,3.40,3.31,2.14
2282,2018,2018-04-14,Santos,Ceará,1.55,3.88,6.12
2283,2018,2018-04-15,América-MG,Sport,1.96,3.27,4.01
2284,2018,2018-04-15,Corinthians,Fluminense,1.62,3.55,6.03


## Merge Both Datasets
Finally the time has come to merge both datasets. 

In [29]:
# There may be problems with timezone, so consider an one day tolerance.
tol = pd.Timedelta(value=1, unit="day")
# merge_asof requires sorting the left variable.
cartola_data = cartola_data.sort_values("date", ignore_index=True)

# First merge home games.
bet_data["club"] = bet_data["home"]
home_games = pd.merge_asof(cartola_data, bet_data, on="date", by="club", direction="nearest", tolerance=tol)#, how="left")

# First merge away games.
bet_data["club"] = bet_data["away"]
away_games = pd.merge_asof(cartola_data, bet_data, on="date", by="club", direction="nearest", tolerance=tol)#, how="left")

cartola_data["win"] = np.where(
    pd.isna(home_games["home_odds"]),
    away_games["away_odds"],
    home_games["home_odds"]
    )

cartola_data["lose"] = np.where(
    pd.isna(home_games["away_odds"]),
    away_games["home_odds"],
    home_games["away_odds"]
)

cartola_data["draw"] = np.where(
    pd.isna(home_games["draw_odds"]),
    away_games["draw_odds"],
    home_games["draw_odds"]
)

cartola_data.head()

,year,name,round,club,position,status,points,mean,price,variation,matches,date,win,lose,draw
0,2018,Juan,2,Flamengo,defender,doubt,0.0,0.5,8.61,-3.39,1.0,2018-04-21,1.41,8.00,4.43
1,2018,Tiago,2,Bahia,defender,null,6.1,1.4,6.48,-1.52,1.0,2018-04-21,2.26,3.34,3.12
2,2018,Ricardo Silva,2,América-MG,defender,null,0.0,0.0,3.00,0.00,0.0,2018-04-21,8.00,1.41,4.43
3,2018,Diego Alves,2,Flamengo,goalkeeper,expected,0.0,1.1,9.71,-3.29,1.0,2018-04-21,1.41,8.00,4.43
4,2018,Lucas Fonseca,2,Bahia,defender,expected,5.5,0.0,7.00,0.00,0.0,2018-04-21,2.26,3.34,3.12


### Check NaNs

Let's check if the merger was success.

In [30]:
# NaNs ratio in each row.
(cartola_data.shape[0] - cartola_data.count()) / cartola_data.shape[0]

year         0.000000
name         0.000000
round        0.000000
club         0.000000
position     0.000000
status       0.000000
points       0.000000
mean         0.000000
price        0.000000
variation    0.000000
matches      0.000000
date         0.000000
win          0.009423
lose         0.009423
draw         0.009423
dtype: float64

The merge was unsuccesful for around 1% of occurences. This is negligible. They can be dropped.

In [31]:
cartola_data = cartola_data.dropna()

### Check Merging
This merger is the most important action so far. It is crutial that it was done correctly, otherwise the model will be completely wrong. It is a good idea to check it.

In [32]:
# Extract a sample from Cartola FC dataframe.
sample = cartola_data.sample(1).iloc[0]

# Find the same game in the betting datase.
bet_sample = bet_data[
    (bet_data["date"] == sample["date"])
    & ((bet_data["home"] == sample["club"]) | (bet_data["away"] == sample["club"]))
].iloc[0]

# Check if drawing odds is the same.
is_odds_equal = sample["draw"] == bet_sample["draw_odds"]
print("Merge was succesfull for this sample.\n")

# Visualize both samples
print("Cartola FC dataset sample:\n", sample, "\n")
print("Betting dataset sample:\n", bet_sample)

Merge was succesfull for this sample.

Cartola FC dataset sample:
 year                        2018
name                     Gabriel
round                         12
club                      Grêmio
position                defender
status                      null
points                         0
mean                           0
price                          1
variation                      0
matches                        0
date         2018-06-13 00:00:00
win                         2.44
lose                        3.11
draw                        2.98
Name: 8164, dtype: object 

Betting dataset sample:
 year                        2018
date         2018-06-13 00:00:00
home                       Sport
away                      Grêmio
home_odds                   3.11
draw_odds                   2.98
away_odds                   2.44
club                      Grêmio
Name: 2393, dtype: object


It confirms that pd.merge_asof was correctly set.

### Keep Only Relevand Features
Some features like date and athlete name were usefull up to know only for merging datasets and debugging operations. However, they are no longer relevant, since those operations are completed.

Also, now we'll work with a single DataFrame and there's no need to call the variable `cartola_data`. I'll rename it to only `data`.

In [33]:
data = cartola_data[[
    "year", 
    "name", 
    "round", 
    "club", 
    "position", 
    "status", 
    "matches", 
    "mean", 
    "price", 
    "variation", 
    "win", 
    "lose", 
    "draw",
    "points",
    ]]
data.head()

,year,name,round,club,position,status,matches,mean,price,variation,win,lose,draw,points
0,2018,Juan,2,Flamengo,defender,doubt,1.0,0.5,8.61,-3.39,1.41,8.00,4.43,0.0
1,2018,Tiago,2,Bahia,defender,null,1.0,1.4,6.48,-1.52,2.26,3.34,3.12,6.1
2,2018,Ricardo Silva,2,América-MG,defender,null,0.0,0.0,3.00,0.00,8.00,1.41,4.43,0.0
3,2018,Diego Alves,2,Flamengo,goalkeeper,expected,1.0,1.1,9.71,-3.29,1.41,8.00,4.43,0.0
4,2018,Lucas Fonseca,2,Bahia,defender,expected,0.0,0.0,7.00,0.00,2.26,3.34,3.12,5.5


### Features Encoding
`position` and `status` are the only non-numerical features. They nedded to be encoded before modelling.

#### Rrders
Orders are fundamental in order to create a meaningfull encoding. The following orders will be used:

* `position`: The most close to the attack a player plays, more likely it is to score more points. The order is based on the further to the attack (coach and then goalkeeper), to the closest (forward).
* `status`: The most minutes a player plays, the most likely it will score points. The order is based on the less likely to play (suspended) to the most likely to play (expected);

In [34]:
# Converto features to category-type.
data["position"] = data["position"].astype("category")
data["status"] = data["status"].astype("category")

# Order
position_order = ["coach", "goalkeeper", "defender", "fullback", "midfielder", "forward"]
status_order = ["suspended", "injured", "null", "doubt", "expected"]
# status_order = ["doubt", "expected"]
data["position"] = data["position"].cat.reorder_categories(position_order)
data["status"] = data["status"].cat.reorder_categories(status_order)

# Transform into codes.
data["position"] = data["position"].cat.codes
data["status"] = data["status"].cat.codes

data.head()

c:\users\mcouto\desktop\personal\palpite\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\mcouto\desktop\personal\palpite\venv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\mcouto\desktop\personal\palpite\venv\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,year,name,round,club,position,status,matches,mean,price,variation,win,lose,draw,points
0,2018,Juan,2,Flamengo,2,3,1.0,0.5,8.61,-3.39,1.41,8.00,4.43,0.0
1,2018,Tiago,2,Bahia,2,2,1.0,1.4,6.48,-1.52,2.26,3.34,3.12,6.1
2,2018,Ricardo Silva,2,América-MG,2,2,0.0,0.0,3.00,0.00,8.00,1.41,4.43,0.0
3,2018,Diego Alves,2,Flamengo,1,4,1.0,1.1,9.71,-3.29,1.41,8.00,4.43,0.0
4,2018,Lucas Fonseca,2,Bahia,2,4,0.0,0.0,7.00,0.00,2.26,3.34,3.12,5.5


### Save Data
Save data and create a read me file explaning the features to other people.

In [35]:
data.to_csv(r"data\data.csv", index=False)

read_me = """# Cartola FC and Events Probabilites.

This dataset mix Cartola FC and game probabilities. Each row represents a player.

## Features

* year: Cartola FC season.
* name: Player name.
* round: Brasileirão round.
* club: Player club.
* position: Player's position. 
    0. Coach
    1. Goalkeeper
    2. Defender
    3. Fullback
    4. Midfielder
    5. Forward
* status: Player's status.
    0. Suspended
    1. Injured
    2. Null
    3. Doubt
    4. Expected
* matches: Amount o matches that the player has played.
* mean: Player's points mean considering only games he has played.
* price: Player's price at selection time.
* variation: How much price changed from last match.
* win: Player's team winning odds.
* lose: Player's team losing odds.
* draw: Player's team drawing odds.
* point: Amount of points the player scored on the match.

Author:

* [Matheus Couto](github.com/matheusccouto)

Sources:

* [github.com/henriquepgomide/caRtola](https://github.com/henriquepgomide/caRtola/tree/master/data)
* [football-data.co.uk/brazil.php](https://www.football-data.co.uk/brazil.php)

"""

with open(r"data\README.md", "w") as file:
    file.write(read_me)

## Model Construction
### Keep Only Numeric Rows
Names, years, rounds and clubs values are useful for finding an occurance, but they are not used for the prediction and should be dropped.

In [36]:
data = data.drop(columns=["year", "name", "round", "club"])

In [37]:
data = data[
    (data["status"] != 0)
    & (data["status"] != 1)
    & (data["status"] != 2)
]
data.head()

,position,status,matches,mean,price,variation,win,lose,draw,points
0,2,3,1.0,0.5,8.61,-3.39,1.41,8.00,4.43,0.0
3,1,4,1.0,1.1,9.71,-3.29,1.41,8.00,4.43,0.0
4,2,4,0.0,0.0,7.00,0.00,2.26,3.34,3.12,5.5
8,3,4,1.0,10.9,9.58,6.58,8.00,1.41,4.43,3.4
11,5,4,1.0,6.4,11.30,1.30,1.41,8.00,4.43,8.4


### Split Dataset
Split the dataset into:

* `x`: features
* `y`: target

In [38]:
x = data.drop("points", axis=1)
x.head()

,position,status,matches,mean,price,variation,win,lose,draw
0,2,3,1.0,0.5,8.61,-3.39,1.41,8.00,4.43
3,1,4,1.0,1.1,9.71,-3.29,1.41,8.00,4.43
4,2,4,0.0,0.0,7.00,0.00,2.26,3.34,3.12
8,3,4,1.0,10.9,9.58,6.58,8.00,1.41,4.43
11,5,4,1.0,6.4,11.30,1.30,1.41,8.00,4.43


In [39]:
y = data[["points"]]
y.head()

,points
0,0.0
3,0.0
4,5.5
8,3.4
11,8.4


#### Train and Test Sets
Split the dataset and target on train and test dataset.

In [40]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2)

### TPOT
TPOT is an automated machined learning tool that uses genetic algorithm to find the most suited model. To know more about TPOT check-out the [documentation](http://epistasislab.github.io/tpot/).

The code from the cell below was genereated automatically by TPOT using the following snippet.
```
tpot_reg = tpot.TPOTRegressor(
    generations=10, population_size=100, verbosity=2, config_dict="TPOT light"
    )
tpot_reg.fit(x_train, y_train)
print(tpot_reg.score(x_test, y_test))
tpot_reg.export('tpot_export.py')
```

It took 1 hour and 14 minutes for TPOT to achieve these results.

In [41]:
# Rename variables to match the way they were declared on the TPOT exported
training_features = x_train
testing_features = x_test
training_target = y_train
testing_target = y_test

In [42]:
import numpy as np
import pandas as pd
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from tpot.builtins import StackingEstimator

# Average CV score on the training set was: -15.414579276900241
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RidgeCV()),
    StandardScaler(),
    KNeighborsRegressor(n_neighbors=93, p=1, weights="distance"),
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

In [43]:
pd.DataFrame({
    "Root Mean Squared Error": sklearn.metrics.mean_squared_error(y_test, results, squared=False),
    "R2": sklearn.metrics.r2_score(testing_target, results),
    }, index=["score"]).transpose()

,score
Root Mean Squared Error,3.990895
R2,0.054152


### Visualize Predictions
Let's see by ourselves some of the predictions.

In [44]:
predict_viz = x_test.copy()
predict_viz["y_test"] = y_test
predict_viz["y_pred"] = results
predict_viz[predict_viz["status"] == 4].sample(5)

,position,status,matches,mean,price,variation,win,lose,draw,y_test,y_pred
13294,1,4,13.0,1.26,6.29,-0.84,6.61,1.51,4.02,-3.0,2.576072
40656,2,4,13.0,3.19,9.85,0.07,2.06,3.81,3.16,1.5,2.924043
49031,2,4,17.0,4.01,10.09,0.00,1.58,5.79,3.83,1.8,4.110572
2253,4,4,3.0,2.10,5.09,1.36,1.74,5.27,3.34,-1.1,2.550865
24860,3,4,19.0,1.40,2.15,-0.37,8.10,1.45,3.96,0.0,1.358792


### Export
Saves model to a file.

In [45]:
joblib.dump(exported_pipeline, 'model.pkl') 

['model.pkl']